In [1]:
# Using all Variables
import re
import pandas as pd
import sklearn

In [2]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

/home/ne04326/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
fadf = pd.read_excel('FA Analysis - 4K with no names - Clean1.xlsx',index = False)
fadf1 = pd.read_excel('DT_score.xlsx',index = False)

In [4]:
fadf2 = pd.read_excel('FA Firm NNM.xlsx',index = False)
fadf3 = pd.merge(fadf, fadf2, left_on='FA Details-GPN', right_on='GPN')

In [5]:
fadf3.head()

,FA Details-GPN,FA Details-I-LOS,FA Details-UBS-LOS,FA Details-Teaming Flag,FA Details-Yrs. Left on EFL (Feb 2018),FA Details-Production Quintile,FA Details-T12 (Feb 2018),FA Details-AuM YE 17,Product Mix (Rev)-Advisory %,Product Mix (Rev)-Alts.,...,Firm YTD (June) NNM,Firm 2017 NNM,Firm 2016 NNM,Firm 2015 NNM,GPN,FAID,YTD (June) NNM,2017 NNM,2016 NNM,2015 NNM
0,43291034,12.05,9.04,0,-0.2,1,1349015.30,1.509757e+08,0.702050,0.007914,...,6914,281,15094,12298,43291034,AJ MS,6914.0,281,15094.0,12298.0
1,80000686,8.08,8.11,1,0.0,3,615707.13,7.922755e+07,0.785594,0.004311,...,8239,5710,7128,3315,80000686,AJ 60,8239.0,5710,7128.0,3315.0
2,43352604,12.02,6.06,1,2.8,5,500575.60,5.665123e+07,0.670920,0.027521,...,527,1828,1953,2868,43352604,AJ MJ,527.0,1828,1953.0,2868.0
3,43298966,16.05,8.11,1,0.2,1,2874005.48,2.946003e+08,0.551391,0.005667,...,42439,497,12886,-4365,43298966,AJ AL,42439.0,497,12886.0,-4365.0
4,43138114,37.07,19.01,1,4.9,1,2015931.35,4.080550e+08,0.102297,0.021113,...,-1878,-7646,9,-3333,43138114,AJ 52,-1878.0,-7646,9.0,-3333.0


In [7]:
fadf3['NNM_quintile'] = pd.qcut(fadf3['2017 NNM'], 4, labels=False)
fadf_final =  pd.concat([fadf1, fadf3[['NNM_quintile']]], axis = 1)

In [8]:
fadf_final.drop(['Segment Index Score (All FA Score)-Client Acquisition Score', 'Segment Index Score (All FA Score)-Delivering the Firm','Segment Index Score (All FA Score)-Planning Score'], axis=1, inplace=True)

In [9]:
fadf_final.head()

,Segment Index Score (All FA Score)-Advisory Score,Segment Index Score (All FA Score)-Client Retention,Segment Index Score (All FA Score)-Marketing Score,Segment Index Score (All FA Score)-Banking Score,NNM_quintile
0,9,7,6.00,4.00,2
1,9,2,5.50,7.75,3
2,7,2,7.75,6.00,2
3,10,1,6.75,10.00,2
4,1,7,5.75,5.00,0


In [10]:
X = fadf_final.iloc[:, 0:4]
Y = fadf_final.iloc[:,4]

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size = 0.3, random_state = 100)

In [12]:
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100,
                        max_depth=4,min_samples_leaf=5)

In [13]:
AA = clf_gini.fit(X_train, Y_train)

In [14]:
with open("DT_score_leftover.dot", "w") as f:
    f = tree.export_graphviz(AA, out_file=f)

In [15]:
name = list(X_train)
name

[u'Segment Index Score (All FA Score)-Advisory Score',
 u'Segment Index Score (All FA Score)-Client Retention',
 u'Segment Index Score (All FA Score)-Marketing Score',
 u'Segment Index Score (All FA Score)-Banking Score']

In [16]:
with open("DT_score_leftover.txt", "w") as f:
    f = tree.export_graphviz(AA, out_file=f, feature_names=name)

In [17]:
y_pred1 = clf_gini.predict(X_train)
y_pred1

array([2, 2, 2, ..., 1, 3, 0])

In [18]:
print "Accuracy is ", accuracy_score(Y_train,y_pred1)*100

Accuracy is  35.460750853242324


In [19]:
y_pred = clf_gini.predict(X_test)
y_pred

array([0, 2, 2, ..., 2, 0, 0])

In [20]:
print "Accuracy is ", accuracy_score(Y_test,y_pred)*100

Accuracy is  33.28025477707006


In [21]:
Imp = pd.DataFrame(clf_gini.feature_importances_)
name1 = pd.DataFrame(name)

In [22]:
Imp1 = pd.concat([name1, Imp], axis=1)
Imp1.columns = ['Var', 'Score']
Imp1 = Imp1.sort_values(by='Score',ascending=False)
Imp1

,Var,Score
1,Segment Index Score (All FA Score)-Client Rete...,0.268694
2,Segment Index Score (All FA Score)-Marketing S...,0.267155
3,Segment Index Score (All FA Score)-Banking Score,0.241471
0,Segment Index Score (All FA Score)-Advisory Score,0.222680


In [23]:
Y_final =  pd.concat([X_test,Y_test], axis = 1)
Y_final.head()

,Segment Index Score (All FA Score)-Advisory Score,Segment Index Score (All FA Score)-Client Retention,Segment Index Score (All FA Score)-Marketing Score,Segment Index Score (All FA Score)-Banking Score,NNM_quintile
221,1,1,4.50,2.25,2
459,8,1,5.25,8.25,0
4034,8,9,0.25,3.75,0
2684,9,8,5.00,3.25,3
2528,2,3,5.25,7.75,0


In [24]:
C1 = Y_final['Segment Index Score (All FA Score)-Marketing Score']  > 4.125
C2 = Y_final['Segment Index Score (All FA Score)-Client Retention'] > 4.5
Y1= Y_final[C1 & C2]

In [25]:
Y2 = Y1.groupby('NNM_quintile').count()
Y2.head()

,Segment Index Score (All FA Score)-Advisory Score,Segment Index Score (All FA Score)-Client Retention,Segment Index Score (All FA Score)-Marketing Score,Segment Index Score (All FA Score)-Banking Score
NNM_quintile,,,,
0,60,60,60,60
1,67,67,67,67
2,91,91,91,91
3,122,122,122,122


In [26]:
 (Y2.ix[3,1])*100 /(Y2.ix[0,1] + Y2.ix[1,1] + Y2.ix[2,1] + Y2.ix[3,1])

/home/ne04326/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.


35

In [45]:
Y2

,Product Mix (Rev)-Alts. ($),Product Mix (Rev)-BANKING Dollar ($),Product Mix (Rev)-EQUITIES ($),Product Mix (Rev)-INSURANCE / ANNUITIES ($),Product Mix (Rev)-LENDING ($),Product Mix (Rev)-MISCELLANEOUS ($),Product Mix (Rev)-MUNICIPALS ($),Product Mix (Rev)-MF and UIT ($),Product Mix (Rev)-OTHER ($),Product Mix (Rev)-TAXABLE FIXED I ($)
NNM_quintile,,,,,,,,,,
0,16,16,16,16,16,16,16,16,16,16
1,9,9,9,9,9,9,9,9,9,9
2,6,6,6,6,6,6,6,6,6,6
3,41,41,41,41,41,41,41,41,41,41


In [46]:
sum = (Y2.ix[0,1] + Y2.ix[1,1] + Y2.ix[2,1] + Y2.ix[3,1])
qunat4 =  Y2.ix[3,1]
print sum
print qunat4 

72
41


In [46]:
ratio = qunat4*100/sum
ratio

56

In [44]:
qunat4.dtype

dtype('int64')

In [45]:
sum.dtype

dtype('int64')